<a href="https://colab.research.google.com/github/EDaviesmathematics/2023_IonQ_Remote/blob/main/drone_route_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dimod
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 39.6 MB/s eta 0:00:00


In [ ]:
import dimod
import numpy as np


#set up to problem
graph = {('start','1'):1,('1','end'):2}
T_max  = 3
num_drones =2


regular_nodes = ['1']
out_nodes = ['start']
in_nodes = ['end']

edges = [edge for edge in graph]

log_d = int(np.ceil(np.log2(num_drones)))
log_T_max =  int(np.ceil(np.log2(T_max)))

nodes = list(set([a for a,b in graph]+[b for a,b in graph]))

node_bin = { a: f'x_{a}'for a in nodes}
edge_bin = {edge: f'e_{edge[0]}_{edge[1]}' for edge in edges}
time_bin = { a: {f't_{a}_{2**i}':2**i for i in range(log_T_max)} for a in nodes}
drone_bin = {a: {f'd_{a}_{2 ** i}': 2 ** i for i in range(log_d)} for a in nodes}

bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)


for a in regular_nodes:
    bqm.add_linear_equality_constraint([(node_bin[a],-1)]+[(edge_bin[edge],1) for edge in edges if a == edge[0]],1,0)
    bqm.add_linear_equality_constraint([(node_bin[a], -1)] + [(edge_bin[edge], 1) for edge in edges if a == edge[1]], 1, 0)


for a in out_nodes:
    bqm.add_linear_equality_constraint([(node_bin[a], -1)] + [(edge_bin[edge], 1) for edge in edges if a == edge[0]], 100, 0)


for a in in_nodes:
    bqm.add_linear_equality_constraint([(node_bin[a], -1)] + [(edge_bin[edge], 1) for edge in edges if a == edge[1]], 100, 0)




for edge in edges:
    a,b = edge

    bqm.add_linear_inequality_constraint(
        [(key,-val) for key, val in time_bin[b].items()] + [(key,val) for key, val in time_bin[a].items()]+[(edge_bin[edge],T_max)],
        1,
        f'time {a} to {b}',
        constant = -T_max+graph[edge],
        ub=0)

for edge in edges:
    a,b = edge
    bqm.add_linear_inequality_constraint(
        [(key,-val) for key, val in drone_bin[a].items()] + [(key,val) for key, val in drone_bin[b].items()]+[(edge_bin[edge],num_drones)],
        1,
        f'd{a}>d{b}',
        constant = -num_drones,
        ub=0)

    bqm.add_linear_inequality_constraint(
        [(key, -val) for key, val in drone_bin[b].items()] + [(key, val) for key, val in drone_bin[a].items()] + [(edge_bin[edge], num_drones)],
        1,
        f'd{b}>d{a}',
        constant = -num_drones,
        ub=0)



for node in out_nodes + in_nodes:
    bqm.fix_variable(node_bin[node],1)

# Solve the QUBO
sampler = dimod.ExactSolver().sample(bqm)
print(sampler.first.sample)
print(bqm.energy(sampler.first.sample))



solution = [key for key, val in sampler.first.sample.items() if val == 1]
edges_on = [key for key in solution if key[0]=='e']
nodes_on = [key for key in solution if key[0] =='x']
times = {a:0 for a in nodes}
for key in solution:
    if key[0]=='t':
        times[key.split('_')[1]] += int(key.split('_')[2])

drone_nodes = {a: 0 for a in nodes}
for key in solution:
    if key[0]=='d':
        drone_nodes[key.split('_')[1]] += int(key.split('_')[2])

print(f'nodes on {nodes_on}')
print(f'edges on {edges_on}')
print(f'node times {times}')
print(f'drone values {drone_nodes}')



{'d_1_1': 0, 'd_end_1': 0, 'd_start_1': 0, 'e_1_end': 1, 'e_start_1': 1, 'slack_d1>dend_0': 0, 'slack_d1>dend_1': 0, 'slack_d1>dstart_0': 0, 'slack_d1>dstart_1': 0, 'slack_dend>d1_0': 0, 'slack_dend>d1_1': 0, 'slack_dstart>d1_0': 0, 'slack_dstart>d1_1': 0, 'slack_time 1 to end_0': 0, 'slack_time 1 to end_1': 0, 'slack_time 1 to end_2': 0, 'slack_time start to 1_0': 0, 'slack_time start to 1_1': 0, 'slack_time start to 1_2': 0, 't_1_1': 1, 't_1_2': 0, 't_end_1': 1, 't_end_2': 1, 't_start_1': 0, 't_start_2': 0, 'x_1': 1}
0.0
nodes on ['x_1']
edges on ['e_1_end', 'e_start_1']
node times {'1': 1, 'end': 3, 'start': 0}
drone values {'1': 0, 'end': 0, 'start': 0}
